# 追踪和可视化实验
模型开发就像驾驶没有窗户的汽车，图表和日志提供了驾驶汽车的窗口。

## 追踪指标（Track metrics)
指标可视化是了解模型在整个模型开发过程的表现的最基本但最强大的方法。

可以在`LightningModule`中使用`self.log`方法记录指标。

In [ ]:
class LitModel(pl.LightningModule):
    def training_step(self, batch, batch_idx):
        value = ...
        self.log("some_value", value)

使用`self.log_dict`一次性记录多个指标。

In [ ]:
values = {"loss": loss, "acc": acc, "metric_n": metric_n}  # add more items if needed
self.log_dict(values)

# 在命令行显示进度条
self.log("train_loss", loss, prog_bar=True)

## 在浏览器中查看
如果需要在浏览器中查看指标，需要使用具有这些功能的实验管理器。

默认情况下，如果Tensorboard可用的话，Lightning会首先使用Tensorboard，否则使用简单的CSV记录器。

In [ ]:
# every trainer already has tensorboard enabled by default (if the dependency is available)
trainer = Trainer()

## 累积指标
在`training_step`中记录的指标会依据时间线依次记录，但是对于验证集和测试集并不需要知道每一步的值，而是希望在一个epoch中计算一个统计性信息。

在`validation_step`和`test_step`中使用`self.log()`会自动累积指标，在一个epoch结束后计算平均值。

In [ ]:
def validation_step(self, batch, batch_idx):
    value = batch_idx + 1
    self.log("average_value", value)
    # 或使用其它统计值
    self.log(..., reduce_fx='sum')

## 配置储存地址
默认储存的地址是当前工作目录，可以通过在`Trainer`中指定`default_root_dir`改变存储位置。

In [ ]:
Trainer(default_root_dir="/your/custom/path")

# 中级教程

`self.log()`只能够记录数值指标，如果需要记录更加丰富的媒体文件，则需要先选择一个`Lightning`支持的记录器。

In [ ]:
from pytorch_lightning import loggers as pl_loggers

tensorboard = pl_loggers.TensorBoardLogger(save_dir="")
trainer = Trainer(logger=tensorboard)

然后可以直接获取这个记录器的API

In [ ]:
def training_step(self):
    tensorboard = self.logger.experiment
    tensorboard.add_image()
    tensorboard.add_histogram(...)
    tensorboard.add_figure(...)

# Wandb


In [ ]:
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(project="MNIST", log_model="all")
trainer = Trainer(logger=wandb_logger)

# log gradients and model topology
wandb_logger.watch(model)

从除了`init`方法的其它任何函数获得wandb的api，然后使用它记录任何它支持的数据。

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        wandb_logger = self.logger.experiment
        fake_images = torch.Tensor(32, 3, 28, 28)

        # Option 1
        wandb_logger.log({"generated_images": [wandb.Image(fake_images, caption="...")]})

        # Option 2 for specifically logging images
        wandb_logger.log_image(key="generated_images", images=[fake_images])

多个实验记录器，可以给`logger`传入一个记录器的列表。

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger

logger1 = TensorBoardLogger()
logger2 = WandbLogger()
trainer = Trainer(logger=[logger1, logger2])

然后通过获取它们的api的方式使用它们

In [ ]:
class MyModule(LightningModule):
    def any_lightning_module_function_or_hook(self):
        tensorboard_logger = self.loggers.experiment[0]
        wandb_logger = self.loggers.experiment[1]

        fake_images = torch.Tensor(32, 3, 28, 28)

        tensorboard_logger.add_image("generated_images", fake_images, 0)
        wandb_logger.add_image("generated_images", fake_images, 0)